Imports

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt

Adatok betöltése, tisztitása

In [ ]:
file_path = "Cseh hírlappiac adattábla.xlsx"
excel_data = pd.ExcelFile(file_path)

df_forgalom = excel_data.parse('Forgalom')
df_bolttorzs = excel_data.parse('Bolttörzs')
df_koltsegek = excel_data.parse('Költségek')
display(df_forgalom.head())
display(df_forgalom.describe())
display(df_forgalom.dtypes)
display(df_bolttorzs.head())
display(df_bolttorzs.describe())
display(df_bolttorzs.dtypes)
display(df_koltsegek.head())
display(df_koltsegek.describe())
display(df_koltsegek.dtypes)

Költség tisztítása

In [ ]:
remaining_columns = df_forgalom.isna().sum()
display(remaining_columns)

numerical_cols = df_forgalom.select_dtypes(include=['float64', 'int64']).columns
non_numerical_cols = df_forgalom.select_dtypes(exclude=['float64', 'int64']).columns

for col in numerical_cols:
    if df_forgalom[col].isna().sum() > 0:
        df_forgalom[col] = df_forgalom[col].fillna(df_forgalom[col].mean())

for col in non_numerical_cols:
    if df_forgalom[col].isna().sum() > 0:
        df_forgalom[col] = df_forgalom[col].fillna("N/A")

remaining_columns = df_forgalom.isna().sum() 
display(remaining_columns)

Bolttörzs tisztítása

In [ ]:
remaining_columns = df_bolttorzs.isna().sum()
display(remaining_columns)

numerical_cols = df_bolttorzs.select_dtypes(include=['float64', 'int64']).columns
non_numerical_cols = df_bolttorzs.select_dtypes(exclude=['float64', 'int64']).columns


for col in numerical_cols:
    if df_bolttorzs[col].isna().sum() > 0:
        df_bolttorzs[col] = df_bolttorzs[col].fillna(df_bolttorzs[col].mean())


for col in non_numerical_cols:
    if df_bolttorzs[col].isna().sum() > 0:
        df_bolttorzs[col] = df_bolttorzs[col].fillna("N/A")

remaining_columns = df_bolttorzs.isna().sum() 
display(remaining_columns)


Forgalom tisztítása

In [ ]:
remaining_columns = df_koltsegek.isna().sum() 
display(remaining_columns)

numerical_cols = df_koltsegek.select_dtypes(include=['float64', 'int64']).columns
non_numerical_cols = df_koltsegek.select_dtypes(exclude=['float64', 'int64']).columns

for col in numerical_cols:
    if df_koltsegek[col].isna().sum() > 0:
        df_koltsegek[col] = df_koltsegek[col].fillna(df_koltsegek[col].mean())

for col in non_numerical_cols:
    if df_koltsegek[col].isna().sum() > 0:
        df_koltsegek[col] = df_koltsegek[col].fillna("N/A")

        
remaining_columns = df_koltsegek.isna().sum() 
display(remaining_columns)


Duplikált sorok törlése

In [ ]:
duplikalt_boltok = df_bolttorzs[df_bolttorzs.duplicated()]
duplikalt_koltsegek = df_koltsegek[df_koltsegek.duplicated]
duplikalt_forgalom = df_forgalom[df_forgalom.duplicated()]


df_bolttorzs = df_bolttorzs.drop_duplicates()
df_koltsegek = df_koltsegek.drop_duplicates()
df_forgalom = df_forgalom.drop_duplicates()

Azokat a mezőket amik érvénytelen értéket vesznek fel az átlagra cseréljük

In [ ]:
oszlopok_check = [
    'Alapterület', 'Létszám', 'Hirlap_regal_db', 'Hirlap_kivul_regal_db'
]

for col in oszlopok_check:
    mask = df_bolttorzs[col] <= 0
    if mask.any():
        avg_val = df_bolttorzs.loc[~mask, col].mean()
        df_bolttorzs.loc[mask, col] = avg_val

In [ ]:
df_forgalom.rename(columns={'Bolt név': 'Boltnév'}, inplace=True)

Aggregálás és korrelációt mátrxi számítása a profitra

In [ ]:
df_gm_ev = (
    df_forgalom.groupby(['Boltnév', 'Év'])['GM'].sum().reset_index()
    )


df_profit_ev = pd.merge(df_gm_ev, df_koltsegek, on=['Boltnév', 'Év'], how='left')
df_profit_ev['Profit'] = df_profit_ev['GM'] + df_profit_ev['Költségek']


df_teljes = pd.merge(df_profit_ev, df_bolttorzs, on='Boltnév', how='left')

df_teljes = df_teljes.sort_values(by=['Profit'], ascending=False).reset_index(drop=True)

df_bolt = (
    df_teljes.groupby('Boltnév')['Profit']
    .agg(Atlag_Profit='mean', Min_Profit='min', Max_Profit='max')
    .reset_index()
    .sort_values(by='Atlag_Profit', ascending=False)
)

top5_boltok = df_bolt.head(5)['Boltnév']
df_top = df_teljes[df_teljes['Boltnév'].isin(top5_boltok)]
display(df_top)

bottom5_boltok = df_bolt.tail(5)['Boltnév']
df_bottom = df_teljes[df_teljes['Boltnév'].isin(bottom5_boltok)]
display(df_bottom)                          

# display(df_teljes.head(10))
# display(df_teljes.tail(10))

pd.set_option('display.max_columns', None)

In [ ]:
# df_korrel = df_teljes.corr(numeric_only=True)

# df_szurt = df_korrel[(df_korrel.abs() > 0.2)]
# df_szurt = df_szurt['Profit'].dropna().drop(labels=['Profit', 'GM', 'Költségek'])
# importatant_cols = df_szurt.index
# display(importatant_cols)

Boltok potenciáljának vizsgálata linreggel

In [ ]:
# df_regresszio = df_teljes.dropna(subset=importatant_cols)


# X = df_regresszio[importatant_cols]
# y = df_regresszio['Profit']

# modell = LinearRegression()
# modell.fit(X, y)


# df_regresszio['Becsült profit'] = modell.predict(X)
# df_regresszio['Eltérés (tény - becsült)'] = df_regresszio['Profit'] - df_regresszio['Becsült profit']
# df_regresszio['Kihasználtság'] = df_regresszio['Profit'] / df_regresszio['Becsült profit']

# display(df_regresszio)


In [ ]:
# r2 = modell.score(X, y)
# r2

In [ ]:
df_korrel = df_teljes.corr(numeric_only=True).drop(labels=['Profit', 'GM', 'Költségek'])

In [ ]:
legnagyobb_r2 = 0
legjobb_threshold = 0

for threshold in np.arange(0.46, 0.00, -0.02):

    df_szurt = df_korrel[(df_korrel.abs() > threshold)]
    df_szurt = df_szurt['Profit'].dropna()
    importatant_cols = df_szurt.index.tolist()
    
    df_regresszio = df_teljes.dropna(subset=importatant_cols)


    X = df_regresszio[importatant_cols]
    y = df_regresszio['Profit']

    modell = LinearRegression()
    modell.fit(X, y)


    df_regresszio['Becsült profit'] = modell.predict(X)
    df_regresszio['Eltérés (tény - becsült)'] = df_regresszio['Profit'] - df_regresszio['Becsült profit']
    df_regresszio['Kihasználtság'] = df_regresszio['Profit'] / df_regresszio['Becsült profit']
    
    r2 = modell.score(X, y)
    
    if r2 > legnagyobb_r2:
        legnagyobb_r2 = r2
        legjobb_threshold = threshold
        
display(legnagyobb_r2, legjobb_threshold)

In [ ]:
df_szurt = df_korrel[(df_korrel.abs() > legjobb_threshold)]
                     
df_szurt = df_szurt['Profit'].dropna()
importatant_cols = df_szurt.index.tolist()

df_regresszio = df_teljes.dropna(subset=importatant_cols)

X = df_regresszio[importatant_cols]
y = df_regresszio['Profit']

modell = LinearRegression()
modell.fit(X, y)

df_regresszio['Becsült profit'] = modell.predict(X)
df_regresszio['Eltérés (tény - becsült)'] = df_regresszio['Profit'] - df_regresszio['Becsült profit']
df_regresszio['Kihasználtság'] = df_regresszio['Profit'] / df_regresszio['Becsült profit']


In [ ]:
kihasznalat_gyenge = df_regresszio[df_regresszio['Kihasználtság'] <= 0.4]
print('--- LEGROSSZABB BOLTOK ---')
display(kihasznalat_gyenge.sort_values(by='Kihasználtság', ascending=True).reset_index(drop=True))


kihasznalat_gyenge_db = kihasznalat_gyenge.shape[0]
print('--- Gyenge kihasználtságú boltok ---')
display(kihasznalat_gyenge_db)

alulteljesito_boltok = df_regresszio[(df_regresszio['Kihasználtság'] > 0.4) & (df_regresszio['Kihasználtság'] <= 0.7)]
display(alulteljesito_boltok.sort_values(by='Kihasználtság', ascending=True).reset_index(drop=True))

print('--- Legjobb boltok ---')
legjobb = df_regresszio.sort_values(by='Kihasználtság', ascending=False).head(20)
display(legjobb)

In [ ]:
df_termek_pivot = df_forgalom.pivot_table(
    index='Boltnév',
    columns='Termékkategória',
    values='GM',
    aggfunc='sum'
)


df_termek_pivot = df_termek_pivot.fillna(0)

df_termek_corr = df_termek_pivot.corr()
display(df_termek_corr)

In [ ]:
df_havi_gm_teljes = (
    df_forgalom.groupby(['Év', 'Hó', 'Termékkategória'])['GM']
    .sum()
    .reset_index()
)

display(df_havi_gm_teljes)


In [ ]:
df_havi_gm_teljes['Dátum'] = pd.to_datetime(df_havi_gm_teljes['Év'].astype(str) + '-' + df_havi_gm_teljes['Hó'].astype(str))

ev_lista = sorted(df_havi_gm_teljes['Év'].unique())
kategoria_lista = df_havi_gm_teljes['Termékkategória'].unique()

fig, axs = plt.subplots(nrows=(len(ev_lista) + 1) // 2, ncols=2, figsize=(14, 10))
axs = axs.flatten()

for i, ev in enumerate(ev_lista):
    adat = df_havi_gm_teljes[df_havi_gm_teljes['Év'] == ev]
    ax = axs[i]
    for kat in kategoria_lista:
        kat_adat = adat[adat['Termékkategória'] == kat]
        ax.plot(kat_adat['Dátum'], kat_adat['GM'], label=kat, marker='o')
    ax.set_title(f"{ev} havi GM trendek")
    ax.set_xlabel('Hónap')
    ax.set_ylabel('GM')
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
df_ev_termkat = (
    df_forgalom.groupby(['Év', 'Termékkategória'])['GM'].sum().reset_index()
)

df_ev_ossz_gm = (
    df_ev_termkat.groupby('Év')['GM']
    .sum()
    .reset_index()
    .rename(columns={'GM': 'Éves össz GM'})
)

df_ev_termkat = pd.merge(df_ev_termkat, df_ev_ossz_gm, on='Év', how='left')
df_ev_termkat['Részesedés (%)'] = 100 * df_ev_termkat['GM'] / df_ev_termkat['Éves össz GM']

df_ev_termkat[df_ev_termkat['Év'] == 2016]
df_ev_termkat[df_ev_termkat['Év'] == 2017]


In [ ]:
df_2016 = df_ev_termkat[df_ev_termkat['Év'] == 2016]
labels_2016 = df_2016['Termékkategória']
sizes_2016 = df_2016['GM']

df_2017 = df_ev_termkat[df_ev_termkat['Év'] == 2017]
labels_2017 = df_2017['Termékkategória']
sizes_2017 = df_2017['GM']

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].pie(sizes_2016, labels=labels_2016, autopct='%1.1f%%', startangle=140)
axes[0].set_title('Termékkategóriák részesedése (2016)')

axes[1].pie(sizes_2017, labels=labels_2017, autopct='%1.1f%%', startangle=140)
axes[1].set_title('Termékkategóriák részesedése (2017)')

plt.tight_layout()
plt.show()